# Craigslist Apartment Scraper

The purpose of this script is to pull apartment listings, characteristics, prices, and reply emails from for rent ads on craigslist. We plan to use this information in order to run an experiment to test the impact of including exclamation points on response rates to inquiries sent.

## Version Notes  
- This version does not include any testing code cells. For testing code cells, refer to v1.5
- Unless you wish to pull new listings, be careful not to run the cell that runs queries in all four cities for the bedroom and price combinations specified
- This version pulls listings from Seattle, Houston, Atlanta, and Chicago

In [2]:
#Import modules
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import re
import numpy as np
import os
import xlsxwriter
import time
import sys
import matplotlib.pyplot as plt

#Set directory for input / output

#DIRECTORY FOR HOME MACHINE
os.chdir('/Users/nwchen24/Desktop/UC_Berkeley/Experiments_and_causality/final_project_github_repo/mids-w241-final/CL_scraper/output')

#DIRECTORY FOR WORK MACHINE
#os.chdir('C:/Users/nchen/Desktop/Nick/UC_Berkeley/experiments_and_causality/final_project_github_repo/mids-w241-final/CL_scraper/output')


## Function to query craigslist  

This function will allow us to specify a price range, the number of bedrooms, and what craigslist site to query (e.g. Denver, SF, NYC, etc.)  

Note that these queries only return a max of 100 results each. Thus, we will want to be specific about the price ranges and bedrooms that we specify so we can maximize the number of listings we are able to capture.

In [3]:
#Define a function to fetch search results
def fetch_search_results(query=None, minAsk=None, maxAsk=None, bedrooms=None, search_URL = None):
    search_params = {key: val for key, val in locals().items() if val is not None}
    if not search_params:
        raise ValueError("No valid keywords")
    resp = requests.get(search_URL, params=search_params, timeout=3)
    resp.raise_for_status()  # <- no-op if status==200
    return resp.content, resp.encoding

## Function to get full URLs and apartment characteristics from query function output  

This function will go through each of the listings found from our query and compile a dataset of URLs and apartment characteristics of all the results from the query. We will use the URLs to get the reply email addresses in a later step.

### Helper functions to get apartment characteristics from each query result  

price, bedrooms, square footage, listing title, posting date / time, and reply linnk

In [4]:
#get href - the relative link to the full apartment listing. These relative links are identified by <a> tags
#and have the class 'result-title hdrlnk'.
def get_href(result):
    href = result.find('a', {'class' : 'result-title hdrlnk'})['href']
    
    if href is None:
        href = np.nan
    
    return href

In [5]:
#get posting ID - These IDs are the data-ID portion of  <a> tags with the class 'result-title hdrlnk'.
def get_posting_ID(result):
    posting_ID = result.find('a', {'class' : 'result-title hdrlnk'})['data-id']
    
    if posting_ID is None:
        posting_ID = np.nan
    
    return posting_ID

In [6]:
#get price - price can be located by <span> tags of class 'result-price'
def get_price(result):
    price = result.find('span', {'class' : 'result-price'})
    
    #convert price to float
    if price is not None:
        price = float(price.text.strip('$'))
        
    else:
        price = np.nan
    
    return price

In [7]:
#get listing title which is identified by the text in the <a> tag with class 'result-title hdrlnk'
def get_title(result):
    title = result.find('a', {'class' : 'result-title hdrlnk'}).text
    
    if title is None:
        title = np.nan
        
    return title


In [8]:
#get the time the listing was posted
def get_posting_date(result):
    posting_date = result.find('time', {'class' : 'result-date'})['datetime']
    
    if posting_date is None:
        posting_date = np.nan
        
    return posting_date

In [9]:
#get bedrooms / sqft which is identified by the <span> tag of class 'housing'
def get_bedrooms_sqft_str(result):
    bedrooms_sqft = result.find('span', {'class' : 'housing'}).text.strip('\n')
    
    if bedrooms_sqft is None:
        price = np.nan
    
    return bedrooms_sqft

def get_bedrooms_sqft(bedrooms_sqft):
    #*******
    #remove the new line characters and white space
    p_1 = re.compile('-|\n|\s')

    bedrooms_sqft = p_1.sub('', bedrooms_sqft)

    #*******
    #get bedrooms
    #compile the regex
    bedroom_p = re.compile(r'\d+(?=br)', re.IGNORECASE)

    #get match in the bedroom / sqft string
    bedroom_m = bedroom_p.match(bedrooms_sqft)

    #get bedrooms
    n_bedrooms = float(bedrooms_sqft[bedroom_m.start(): bedroom_m.end()])

    #*******
    #get square footage
    #remove bedrooms
    bedrooms_sqft = bedrooms_sqft[bedroom_m.end() + 2:]

    #compile the regex
    sqft_p = re.compile(r'\d+(?=ft)', re.IGNORECASE)

    #get match in the square footage string
    sqft_m = sqft_p.match(bedrooms_sqft)

    #get square footage
    try:
        sqft = float(bedrooms_sqft[sqft_m.start():sqft_m.end()])
    
    except AttributeError:
        sqft = np.nan
    
    return n_bedrooms, sqft


### Function to compile all apartment characteristics

In [10]:
def compile_listing_URLs(query_result, base_URL, reply_string, city):
    #parse the results of the query
    html = bs4(query_result, 'html.parser')

    #get all individual apartments from the query
    apt_results = html.find_all('p', attrs={'class' : 'result-info'})

    #initialize a list to contain all of the URLs that resulted from the query
    apts_results_df = pd.DataFrame(columns = ('city', 'base_URL', 'href','posting_ID', 'Listing_Title', 'Bedrooms', 'Sqft', 'Price', 'Posting_Date'))
   
    #Looop through all of the tags containing the apartments and get the addresses of those individual results.
    for apt in range(len(apt_results)):
        #use helper functions to get characteristics
        href = get_href(apt_results[apt])
        posting_ID = get_posting_ID(apt_results[apt])
        title = get_title(apt_results[apt])
        bedrooms_sqft_str = get_bedrooms_sqft_str(apt_results[apt])
        bedrooms, sqft = get_bedrooms_sqft(bedrooms_sqft_str)
        price = get_price(apt_results[apt])
        posting_date = get_posting_date(apt_results[apt])
        #populate the result dataframe with the characteristics
        apts_results_df.loc[apt] = [city, base_URL, href, posting_ID, title, bedrooms, sqft, price, posting_date]

    #construct full URL for the listing
    apts_results_df['full_URL'] = apts_results_df.apply(lambda row: row['base_URL'] + row['href'], axis = 1)
    
    #construct reply URL for the listing
    apts_results_df['Reply_contact_info_link'] = apts_results_df.apply(lambda row: row['base_URL'] + reply_string + row['posting_ID'].strip('.html'), axis = 1)
    
    #delete base URL and href columns
    del apts_results_df['base_URL']
    del apts_results_df['href']
    
    return apts_results_df

# Operationalizing Phase  

This phase will incorporate the ability to run the scraper across a selection of cities and bedroom and price range specifications

## City to Craigslist URLs Dictionary  
- If we want to expand the scope of our project to additional cities, this is the place to do so.

In [12]:
#create city list
cities = ['seattle', 'houston', 'chicago', 'sandiego']

#set base craigslist URLs
base_URLs = ['https://seattle.craigslist.org', 'https://houston.craigslist.org', 'https://chicago.craigslist.org', \
             'https://sandiego.craigslist.org']

#set search URLS to feed to query function
search_URLs = ['https://seattle.craigslist.org/search/apa', 'https://houston.craigslist.org/search/apa', \
               'https://chicago.craigslist.org/search/apa', 'https://sandiego.craigslist.org/search/apa']

#set reply strings which are intermediate strings between the base URL and the posting ID to access the page where
#reply emails are found
reply_strings = ['/reply/sea/apa/', '/reply/hou/apa/', '/reply/chi/apa/', '/reply/sdo/apa/']

#create dataframe with all of these pieces of information
city_to_URL_dict = {'base_URL' : base_URLs, 'search_URL' : search_URLs, 'reply_string' : reply_strings}

city_to_URL_df = pd.DataFrame(city_to_URL_dict, index = cities)

## Pull data for bedroom, city, price combinations - DO NOT RUN UNLESS YOU ACTUALLY WANT TO PULL NEW LISTINGS 

- Export results for each city to csv files

In [13]:
#Look at $500 price bucket increments for each number of bedrooms
min_prices = np.arange(1000, 4000, 500).tolist()

#Look at 1, 2, 3 bedroom apartments
bedrooms = [1, 2]

#initialize counter which we will use to populate the query result dataframe
counter = 0

#loop over cities
for city in city_to_URL_df.index:
    #initialize empty dataframe to hold query results
    query_results_df = pd.DataFrame(columns = ('city', 'min_price', 'max_price', 'bedrooms', 'query_html'))

    #loop over number of bedrooms
    for bedroom in bedrooms:
        #loop over the min prices
        for price in min_prices:
            #set start time
            start_time = time.time()

            #ping CL server to get query results
            query_results, query_encoding = fetch_search_results(query = None, minAsk = price, maxAsk = price + 500, bedrooms = bedroom,\
                                 search_URL = city_to_URL_df.loc[city, 'search_URL'])

            if query_results is not None:
                print "Query Success. City: " + city + ", Bedrooms: " + str(bedroom) + ", Price Range: " + str(price) +\
                "-" + str(price + 500)            

            else:
                print "No Results Found. City: " + city + ", Bedrooms: " + str(bedroom) + ", Price Range: " + str(price) +\
                "-" + str(price + 500)            


            #append result (which is a string) to query results dataframe
            query_results_df.loc[counter] = [city, price, price + 500, bedroom, query_results]

            #increment counter
            counter += 1

            #incorporate delay drawn from normal distribution centered around one minute to hopefully make
            #queries appear more human like
            delay = abs(np.random.normal(25, 10))
            time.sleep(delay)
            print "Delay: " + str(time.time() - start_time)

    #write the results for each city to csv
    query_results_df.to_csv(city + '_query_results_output_3-17-17.csv', sep='\t', encoding='utf-8')



Query Success. City: seattle, Bedrooms: 1, Price Range: 1000-1500
Delay: 13.7592389584
Query Success. City: seattle, Bedrooms: 1, Price Range: 1500-2000
Delay: 30.7660520077
Query Success. City: seattle, Bedrooms: 1, Price Range: 2000-2500
Delay: 22.3494670391
Query Success. City: seattle, Bedrooms: 1, Price Range: 2500-3000
Delay: 36.4256830215


KeyboardInterrupt: 

# Get listing information we need from the query results

In [11]:
#First, read the listings back from csv
file_names = []

#compile file names to read
for city in city_to_URL_df.index:
    file_names.append(city + '_query_results_output_3-17-17.csv')

#initialize query results df
query_results_comb_df = pd.DataFrame(columns = ('Unnamed: 0', 'city', 'min_price', 'max_price', 'bedrooms', 'query_html'))

#read each output file for each city and append to a combined dataframe
for filename in file_names:
    query_results_comb_df = query_results_comb_df.append(pd.read_csv(filename, sep='\t', encoding='utf-8'))

#delete columns we don't want and reset index
del query_results_comb_df['Unnamed: 0']
query_results_comb_df = query_results_comb_df.reset_index()
del query_results_comb_df['index']


## Loop through combined dataframe to get individual listings

In [12]:
#initialize listing results df
combined_listings_df = pd.DataFrame(columns = ('city', 'posting_ID', 'Listing_Title', 'Bedrooms', 'Sqft', 'Price', 'Posting_Date'))

#loop through query results
for i in range(query_results_comb_df.shape[0]):
    #get city
    city_to_use = query_results_comb_df.loc[i, 'city']
    #get the intermediate reply string to feed to function that gets individual listings
    reply_string_to_use = city_to_URL_df.loc[city_to_use, 'reply_string']
    #get base URL to feed to function that gets individual listings
    base_URL_to_use = city_to_URL_df.loc[city_to_use, 'base_URL']
    #get the query result html code to feed to function that gets individual listings
    query_result_html = query_results_comb_df.loc[i, 'query_html']
    
    #Get individual listings from query results
    query_results_df_intermed = compile_listing_URLs(query_result = query_result_html, base_URL = base_URL_to_use,\
                                                     city = city_to_use, reply_string = reply_string_to_use)
    
    #append the results for each bedroom, city, price combination to the combined dataframe
    combined_listings_df = combined_listings_df.append(query_results_df_intermed)

In [13]:
#Save combined listing dataset to csv
#re-order columns
combined_listings_df = combined_listings_df[['Listing_Title', 'Posting_Date', 'city', 'posting_ID', 'full_URL', 'Bedrooms',\
                                            'Sqft', 'Price', 'Reply_contact_info_link']]

#Add a column where mechanical turk can fill in reply email address
combined_listings_df['reply_email_TO_BE_FILLED_IN'] = ''

#export to CSV
combined_listings_df.to_csv('Combined Scraper Listing Results 3-17-17.csv', sep=',', encoding='utf-8')

In [14]:
combined_listings_df.head()

combined_listings_df.shape

(5760, 10)

In [29]:
#Select a random sample from the query results.
#Also assign each to treatment group

#First, initialize the dataframe to hold the results
test_sample_df = pd.DataFrame(columns = ('city', 'Listing_Title', 'Posting_Date', 'Bedrooms', 'Sqft', 'Price',\
                           'posting_ID', 'treatment_assignment', 'full_URL', 'Reply_contact_info_link',\
                                         'reply_email_TO_BE_FILLED_IN', 'Apartment_Complex_Flag'))
#create vector of possible treatment assignments
#create vector of possible treatment assignments
treatments = ['John_Control'] * 15 + ['John_Treat_Low'] * 15 + ['John_Treat_High'] * 15 \
                + ['Jane_Control'] * 15 + ['Jane_Treat_Low'] * 15 + ['Jane_Treat_High'] * 15
    
for city in cities:
    #Instantiate city DF
    city_df = pd.DataFrame(columns = ('city', 'Listing_Title', 'Posting_Date', 'Bedrooms', 'Sqft', 'Price',\
                           'posting_ID', 'treatment_assignment', 'full_URL', 'Reply_contact_info_link',\
                                         'reply_email_TO_BE_FILLED_IN', 'Apartment_Complex_Flag'))

    for bedroom in [1,2]:
        #Take a sample of five listings for each city and bedroom combination
        intermed_df = combined_listings_df.loc[(combined_listings_df.city == city)\
                                               & (combined_listings_df.Bedrooms == bedroom),:].sample(n = 90)
 
        #Assign treatment
        intermed_df['treatment_assignment'] = intermed_df['treatment_assignment'] =\
            np.random.choice(treatments, replace = False, size = 90)
        
        #Append results to city df
        city_df = city_df.append(intermed_df)
        
        #shuffle listings within city df
        city_df = city_df.sample(frac = 1)
        
        #re-order columns
        city_df = city_df[['city', 'Listing_Title', 'Posting_Date', 'Bedrooms', 'Sqft', 'Price',\
                           'posting_ID', 'treatment_assignment', 'full_URL', 'Reply_contact_info_link',\
                                         'reply_email_TO_BE_FILLED_IN', 'Apartment_Complex_Flag']]

        city_df = city_df.sort_values(by = 'treatment_assignment')
        
        #export each city
        city_df.to_csv(city + ' Listings For Data Pull 3-17-17.csv', sep=',', encoding='utf-8')
    
    #append results to data frame
    test_sample_df = test_sample_df.append(city_df)

print test_sample_df.shape

#test_sample_df.to_csv('Listings For Data Pull 3-17-17.csv', sep=',', encoding='utf-8')

(720, 12)


## Randomization Checks

In [34]:
#Average square footage and price for each city
by_city_avgs = test_sample_df.groupby(by = ['city', 'treatment_assignment'])['Sqft', 'Price', 'Bedrooms'].mean()

by_city_avgs

#combined_listings_df.head()

Sqft        Price  Bedrooms
city     treatment_assignment                                    
chicago  Jane_Control          1027.666667  2136.066667       1.5
         Jane_Treat_High       1223.444444  2095.433333       1.5
         Jane_Treat_Low        1032.750000  2055.466667       1.5
         John_Control           989.750000  1790.433333       1.5
         John_Treat_High       1059.333333  2251.400000       1.5
         John_Treat_Low        1051.250000  1859.066667       1.5
houston  Jane_Control           961.964286  2083.333333       1.5
         Jane_Treat_High       1078.181818  2373.033333       1.5
         Jane_Treat_Low        1037.950000  2114.600000       1.5
         John_Control          1005.208333  2153.933333       1.5
         John_Treat_High       1061.782609  2134.400000       1.5
         John_Treat_Low         973.695652  2180.766667       1.5
sandiego Jane_Control           870.210526  2045.500000       1.5
         Jane_Treat_High        893.476190  1872.566667       1.5
         Jane_Treat_Low         878.304348  1939.633333       1.5
         John_Control           924.913043  2005.633333       1.5
         John_Treat_High        910.851852  2123.100000       1.5
         John_Treat_Low         874.818182  1811.033333       1.5
seattle  Jane_Control           871.206897  2189.233333       1.5
         Jane_Treat_High        920.600000  2329.400000       1.5
         Jane_Treat_Low         875.392857  2155.733333       1.5
         John_Control           908.344828  2373.900000       1.5
         John_Treat_High        873.074074  2134.100000       1.5
         John_Treat_Low         891.413793  2206.733333       1.5

In [66]:
#Test for statistically significant differences in price, sqft, bedrooms for each treatment
import scipy.stats as stats

#create groups for price, bedrooms, and squarefootage
grps = pd.unique(test_sample_df.treatment_assignment.values)
price_data = {grp:test_sample_df['Price'][test_sample_df.treatment_assignment == grp] for grp in grps}
sqft_data = {grp:test_sample_df['Sqft'][test_sample_df.treatment_assignment == grp] for grp in grps}
bedroom_data = {grp:test_sample_df['Bedrooms'][test_sample_df.treatment_assignment == grp] for grp in grps}



#conduct one way ANOVAs

#Price
F, p = stats.f_oneway(price_data['Jane_Control'], price_data['Jane_Treat_High'], price_data['Jane_Treat_Low'],\
                     price_data['John_Control'], price_data['John_Treat_High'], price_data['John_Treat_Low'])
print 'Price ANOVA p-value: ' + str(p)

#Sqft
F, p = stats.f_oneway(sqft_data['Jane_Control'], sqft_data['Jane_Treat_High'], sqft_data['Jane_Treat_Low'],\
                     sqft_data['John_Control'], sqft_data['John_Treat_High'], sqft_data['John_Treat_Low'])
print 'Square Footage ANOVA p-value: ' + str(p)

#Bedrooms
F, p = stats.f_oneway(bedroom_data['Jane_Control'], bedroom_data['Jane_Treat_High'], bedroom_data['Jane_Treat_Low'],\
                     bedroom_data['John_Control'], bedroom_data['John_Treat_High'], bedroom_data['John_Treat_Low'])
print 'Bedroom ANOVA p-value: ' + str(p)




Price ANOVA p-value: 0.639251057188
Square Footage ANOVA p-value: nan
Bedroom ANOVA p-value: 1.0


In [ ]:

# Etract individual groups
Jane_Control = test_sample_df[groups["Jane_Control"]]
Jane_Treat_High = test_sample_df[groups["Jane_Treat_High"]]
Jane_Treat_Low = test_sample_df[groups["Jane_Treat_Low"]]
John_Control = test_sample_df[groups["John_Control"]]
John_Treat_High = test_sample_df[groups["John_Treat_High"]]
John_Treat_Low = test_sample_df[groups["John_Treat_Low"]]

# Perform the ANOVA
stats.f_oneway(Jane_Control, Jane_Treat_High, Jane_Treat_Low, John_Control, John_Treat_High, John_Treat_Low)
